# antlr

Thus begins the day that my notebook frameworks starts absorbing other frameworks.



## introduction



Why do this. Ultimately, it creates a shortcut for including interpreters for any language in these notebooks. Less reliance on other code, and also converting a library from one framework to another.



In [ ]:
// placeholder for readme.md?


## multilungual abstract syntax trees

This is great day for translating between languages.


### select antlr tree

Using antlr templates because look at all the glorious grammars posted right there on it's main Github page! https://github.com/antlr/grammars-v4



#### the code

select antlr tree?


In [ ]:
import fs from 'fs'
//import path from 'path'
import antlr from 'antlr4'
const {selectDom} = importer.import('select tree')

//TODO: fix this with own transpiler because no one is going to share their hard work on StringTemplate4 code
/*
function toString(ctx, subctx) {
    var ast = (subctx || ctx)
    if(typeof ast.ownerDocument !== 'undefined') {
        ast = htmlToTree(ast)
    }
    //escodegen.attachComments(ast, ast.comments, ast.tokens, ast.whitespace)
    return escodegen.generate(ast, {
        comment: true,
        tokens: true,
        whitespace: true,
        format: {indent: {style: '    '}, quotes: 'single'}
    })
}
*/

let extToLang
let treeToHtml
let getParser
async function selectCode(descriptor, code, language) {
    if(!extToLang)
        extToLang = await importer.import('ext to lang')
    if(!treeToHtml)
        treeToHtml = (await importer.import('antlr to html')).treeToHtml
    if(!getParser)
        getParser = await importer.import('get antlr tool')
    if(typeof code === 'function') {
        code = code.toString()
    }
    if(typeof code === 'string') {
        if(fs.existsSync(code)) {
            if(typeof language === 'undefined') {
                // TODO: use language file extension from media-server code module gheshi support
                language = await extToLang(code)
            }
            code = fs.readFileSync(code).toString('utf8')
        }
        var {lexer, parser} = await getParser(language)
        try {
            const chars = new antlr.InputStream(code);
            const lexerStream = new lexer(chars);
            const tokens = new antlr.CommonTokenStream(lexerStream);
            const parserStream = new parser(tokens);
            parserStream.input = code
            const tree = parserStream[parserStream.ruleNames[0]]();
            code = selectDom(['//' + parserStream.ruleNames[0]], treeToHtml(tree))[0]
        } catch(e) {
            console.log(e)
            throw new Error('Could not parse \n' + code)
        }
    }

    return selectDom(descriptor, code)
}

export default selectCode


#### test select jison on some quake 3 C code?



In [ ]:
const fs = require('fs')
const path = require('path')
//const importer = require('../Core')
const getTypes = importer.import('get c types')
//var antlrToESTree = importer.import('antlr tree to es')

async function testAntlr() {
    let codeCell = importer.interpret('rgb 2 lab')
    var result = getTypes(codeCell.source.join(''))
    console.log(result)
    var result = await (await selectCode)(['//functionDefinition', 
     './declarator//declaratorid/@strdata'], path.join(__dirname, '../mouse.m'), 'cpp')
    //var modified = antlrToESTree(result)
    fs.writeFileSync('tree.json', JSON.stringify(result, null, 2))
    return result
}

module.exports = testAntlr

if(typeof $$ !== 'undefined') {
    $$.async()
    testAntlr()
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


## antlr tools

Creating lexer and parser from .g4 files. Converting XPath to trees. Convert antlr Tool classes to nodejs so we can load grammars on the fly.



### get antlr language



#### the code

get antlr language?

get antlr tool?


In [ ]:
import fs from 'fs'
import path from 'path'

const PARSER_PATH = path.join(__dirname, '../Resources/Parsers')
const LANGUAGES = []
const GRAMMARS = []

function findGrammars(directory) {
    let grammars = fs.readdirSync(path.join(PARSER_PATH, directory))
    for(let j = 0; j < grammars.length; j++) {
        if(grammars[j][0] == '.' || grammars[j].match('examples')) {
            continue
        }
        if(grammars[j].endsWith('.g4')) {
            let language = path.basename(grammars[j], path.extname(grammars[j]))
                .replace('Lexer', '').replace('Parser', '')
            if(!LANGUAGES.includes(directory)) {
                LANGUAGES.push(directory)
            }
            if(!LANGUAGES.includes(language)) {
                LANGUAGES.push(language)
            }

            GRAMMARS.push(path.join(directory, grammars[j]))
        } else if (fs.statSync(path.join(PARSER_PATH, directory, grammars[j])).isDirectory()) {
            findGrammars(path.join(directory, grammars[j]))
        }
    }
}

let languages = fs.readdirSync(PARSER_PATH)
for(let i = 0; i < languages.length; i++) {
    if(languages[i][0] == '.') {
        continue
    }
    if(!fs.statSync(path.join(PARSER_PATH, languages[i])).isDirectory()) {
        continue
    }
    findGrammars(languages[i])
}

async function getParser(language) {
    for(let i = 0; i < GRAMMARS.length; i++) {
        if(GRAMMARS[i].match(language + '/') || GRAMMARS[i].match('/' + language)) {
            let lang = path.basename(GRAMMARS[i], path.extname(GRAMMARS[i]))
                .replace('Lexer', '').replace('Parser', '')
            let lexerPath = path.join(PARSER_PATH, path.dirname(GRAMMARS[i]), lang + 'Lexer.js')
            let parserPath = path.join(PARSER_PATH, path.dirname(GRAMMARS[i]), lang + 'Parser.js')
            console.log(language, lexerPath, parserPath)
            if(!fs.existsSync(lexerPath) || !fs.existsSync(parserPath)) {
                continue
            }
            let codeCell = `
import lexer from '${lexerPath}'
import parser from '${parserPath}'
export default {lexer, parser}
`
            let module = await importer.makeESModule(codeCell, __filename + '<inline>', global)
            return module
        }
    }
    return {}
}

export default getParser




### antlr to html

antlr to html?



In [ ]:
var importer = require('../Core');
var {selectDom} = importer.import('select tree')

//TODO: remove \u200b characters
var specialChars = (str) => {
    var special = {
        '&':'&amp;',
        '<':'&lt;',
        '>':'&gt;',
        '"':'&quot;',
        "'":'&apos;'
    }
    Object.keys(special).forEach(s => {
        str = (str || '').replace(new RegExp(s, 'ig'), special[s])
    })
    return str
}

function filterObject(obj) {
  let result = {}
  for(var i in obj) {
    if(i == 'children' || i == 'parentCtx') {
      continue
    }
    result[i] = obj[i]
  }
  return result
}

function treeToStr(statement, parent, parser) {
    var {type, ruleIndex, start, stop, exception, strdata} = statement;

    if(typeof start == 'object') {
        statement['start'] = start.start
        statement['line'] = start.line
    }
    if(typeof stop == 'object') {
        statement['stop'] = stop.stop
    }

    if(!parser)
        parser = (statement.parentCtx ? statement.parentCtx.parser : statement.parser)


    if(typeof type == 'number') {
        type = parser.ruleNames[type]
    } else
    if(typeof ruleIndex == 'number') {
        type = parser.ruleNames[ruleIndex]
        statement.type = type
    }

    if(exception) {
        return ''
    }

    if(!strdata && statement.start && statement.stop && statement.type) {
        statement.strdata = parser.input.substring(statement.start, statement.stop + 1)
    }

    var result = ``, attrs = ``;
    var isList = true;
    for(var i in statement) {
        if(parseInt(i) + '' === i + '') {
            result += treeToStr(statement[i], parent, parser)
            continue;
        }
        if(i == 'parentCtx' || i == 'parser' || i[0] == '_' || i == 'exception' || i == 'source') {
            continue
        }

        if(i == 'strdata') {
            statement[i] = statement[i].split('\n')[0]
        }

        if(i == 'type' && typeof statement[i] == 'number') {
            statement[i] = parser.ruleNames[statement[i]]
        }

        isList = false;
        var jsType = typeof statement[i];
        if(jsType === 'object' && statement[i]) {
            jsType = Object.getPrototypeOf(statement[i]).constructor.name;
        }

        if(i == 'symbol' && jsType == 'vt') {
            continue
        }

        // if the property is an Object, print out as a child
        // TODO: replace this type property when converting other trees
        //   should be like getEl() treeToStr(() => el.type)
        if(statement[i] && typeof statement[i].type !== 'undefined'
           // or if the property is an Array,
           //   print them out as child elements
           || (Array.isArray(statement[i]) /* && statement[i].length
           && typeof statement[i][0].type !== 'undefined' */)
        ) {
            // print out the other keys just for fun matching
            attrs +=  `
${i}="" ${i}-type="${jsType}"`;
            result += `
${treeToStr(statement[i], i, parser)}`
            // output as child element and <-attr> elements
            //   for posterity, only child elements are return on the select,
            //   but attr elements can still be matched
        } else {
            // if the property is not an object, i.e. native types
            if(typeof statement[i] == 'object' && statement[i] !== null) {
                attrs +=  `
${i}="${specialChars(JSON.stringify(filterObject(statement[i])))}" ${i}-type="${jsType}"`
            } else if(typeof statement[i] !== 'string') {
                attrs +=  `
${i}="${specialChars(JSON.stringify(statement[i]))}" ${i}-type="${jsType}"`
            } else {
                attrs +=  `
${i}="${specialChars(statement[i])}" ${i}-type="${jsType}"`
            }
        }
    }
    
    return isList ? result : `
<${type}${attrs}${parent ? `
 parent-attr="${parent}"`: ``}>${result}</${type}>`
}

function treeToHtml(tree) {
    var body = treeToStr(tree);
    require('fs').writeFileSync('text.html', body)
    return selectDom('//BODY', '<BODY>' + body + '</BODY>');
}

module.exports = {treeToHtml}


### antlr tree visitor



#### the code

antlr tree visitor?


In [ ]:
var {Interval} = require('antlr4ts/misc')

function typeToString(node) {
    if(node === 'object' && node) {
        return Object.getPrototypeOf(node).constructor.name;
    } else {
        return typeof node;
    }
}

function getGenericToken(parser, token) {
    if(!token) return
    return {
        type: 'TerminalNode',
    //    type2: parser.ruleNames[child._stop._type],
        value: token.source.stream.getText(Interval.of(token.start, token.stop)),
        range: [token.start, token.stop]
    }
}

function getGenericContext(ctx) {
    var exclude = [
        'constructor',
        'ruleIndex',
        'enterRule',
        'exitRule',
        'accept'
    ]
    var allTokens = Object.getOwnPropertyNames(Object.getPrototypeOf(ctx))
        .filter(t => exclude.indexOf(t) === -1)
    return allTokens.reduce((obj, cur) => {
        obj[cur] = ctx[cur]()
        return obj
    }, {})
}


function getGenericVisitor({parser, visitor}) {
    function GenericVisitor() {
    //    visitor.apply(this, arguments)
    }
    GenericVisitor.prototype = Object.create(visitor.prototype)
    GenericVisitor.prototype.constructor = GenericVisitor
    GenericVisitor.prototype.defaultResult = function () {
        return {children: []}
    }
    GenericVisitor.prototype.aggregateResult = (ctx, child) => {
        if(!Array.isArray(ctx)) ctx = ctx.children
        ctx[ctx.length] = child
        return ctx
    }
    GenericVisitor.prototype.visit = function (tree) {
        return tree.accept(this)
    }
    // TODO: combine range with return valuess
    GenericVisitor.prototype.visitTerminal = function (node) {
        return getGenericToken(parser, node._symbol)
    }
    GenericVisitor.prototype.visitChildren = function (ctx) {
        var result = visitor.prototype.visitChildren.apply(this, [ctx])
        return {
            type: Object.getPrototypeOf(ctx).constructor.name,
            children: result
        }
    }
    return {
        GenericVisitor
    }
}

module.exports = getGenericVisitor


#### test manipulating different types of structures


In [ ]:

// TODO: move this in to CPP visitor above is our GenericVisitor
GenericVisitor.prototype.visitIdexpression = function (ctx) {
//    var id = getGenericContext.apply(this, [ctx])
    return {
        type: 'Identifier',
        name: this.visitChildren(ctx).flat().pop()
    }
}
GenericVisitor.prototype.visitMemberdeclaration = function (ctx) {
    var obj = getGenericContext.apply(this, [ctx])
    return {
        type: 'Property',
        kind: obj.declspecifierseq.accept(this).flat(4).pop(), // get, set, init
        key: obj.memberdeclaratorlist.accept(this).flat(6).pop(),
    //    computed: boolean,
        value: null, // TODO: default value
        method: false,
    //    shorthand: boolean,
    }
}
GenericVisitor.prototype.visitTrailingtypespecifier = function (ctx) {
    return {
        type: 'Type',
        name: this.visitChildren(ctx).flat(4).pop()
    }
}
GenericVisitor.prototype.visitTypespecifier = function (ctx) {
    return this.visitChildren(ctx).flat().pop()
}
GenericVisitor.prototype.visitEqualityexpression = function (ctx) {
    var obj = getGenericContext.apply(this, [ctx])
    if(!obj.NotEqual && !obj.Equal) return this.visitChildren(ctx)
    return {
        type: 'BinaryExpression',
        operator: (obj.NotEqual || obj.Equal).accept(this),
        left: obj.equalityexpression.accept(this),
        right: obj.relationalexpression.accept(this),
    }
}
GenericVisitor.prototype.visitClassspecifier = function (ctx) {
    var obj = getGenericContext.apply(this, [ctx])
    return {
        type: 'ClassExpression',
        id: null,
        superClass: obj.classhead.accept(this).flat(3).pop(),
        body: obj.memberspecification.accept(this),
    }
}
GenericVisitor.prototype.visitPtrdeclarator = function (ctx) {
    var obj = getGenericContext.apply(this, [ctx])
    return 
}
GenericVisitor.prototype.visitSimpledeclaration = function (ctx) {
    var obj = getGenericContext.apply(this, [ctx])
}
GenericVisitor.prototype.visitFunctiondefinition = function (ctx) {
    var obj = getGenericContext.apply(this, [ctx])
    var kind = getGenericContext.apply(this, [obj.declspecifierseq])
    console.log(kind)
    var declarator = obj.declarator.accept(this)
    return {
        type: 'Function',
    //    id: ,
    //    params: ,
        body: obj.functionbody.accept(this),
        kind: ,
    //    generator: boolean,
    //    async: boolean,
    //    expression: boolean,
        static: kind[0].flat(3).pop() === 'static',
    //    returnType: returnType ? returnType.accept(this) : returnType,
    //    void: ctx.declspecifierseq().accept(this) === 'void',
    }
}
GenericVisitor.prototype.visitParameterdeclaration = function (ctx) {

}
GenericVisitor.prototype.visitDeclspecifierseq = function (ctx) {

}
GenericVisitor.prototype.visit = function (ctx) {
    console.log(Object.getPrototypeOf(ctx))
    var visit = Object.getPrototypeOf(ctx).constructor.name
        .replace(/Context$/, '')
    if(typeof this['visit' + visit] === 'function') {
        return this['visit' + visit](ctx)
    }
    var obj = getGenericContext.apply(this, [ctx])
    return Object.keys(obj).reduce((ret, cur) => {
        ret[cur] = obj ? obj.accept(this) : obj
    }, {})
}
GenericVisitor.prototype.visitTranslationunit = function (ctx) {
    var obj = getGenericContext.apply(this, [ctx])
    return {
        type: 'Program',
        body: obj.declarationseq.accept(this)
    }
}
GenericVisitor.prototype.visitDeclarationseq = function (ctx) {
    var obj = getGenericContext.apply(this, [ctx])
    if(!obj.declarationseq) return obj.declaration.accept(this)
    return [...obj.declarationseq.accept(this), ...obj.declaration.accept(this)]
}
GenericVisitor.prototype.visitFunctiondefinition = function (ctx) {
    var obj = getGenericContext.apply(this, [ctx])
    var decl = getGenericContext.apply(this, [obj.declarator])
    var ptr = getGenericContext.apply(this, [decl.ptrdeclarator])
    var noptr = getGenericContext.apply(this, [ptr.noptrdeclarator])
    return {
        type: 'Function',
        id: getGenericContext.apply(this, [noptr.noptrdeclarator]).declaratorid.accept(this),
        params: noptr.parametersandqualifiers.accept(this),
        body: obj.functionbody.accept(this),
    //    returnType: returnType ? returnType.accept(this) : returnType,
    //    void: ctx.declspecifierseq().accept(this) === 'void',
    }
}
GenericVisitor.prototype.visitDeclaratorid = function (ctx) {
    return getGenericContext.apply(this, [ctx]).idexpression.accept(this)
}

GenericVisitor.prototype.visitFunctionbody = function (ctx) {
    return getGenericContext.apply(this, [ctx]).compoundstatement.accept(this)
}
GenericVisitor.prototype.visitCompoundstatement = function (ctx) {
    return {
        type: 'BlockStatement',
        body: getGenericContext.apply(this, [ctx]).statementseq.accept(this)
    }
}
GenericVisitor.prototype.visitStatementseq = function (ctx) {
    var obj = getGenericContext.apply(this, [ctx])
    if(!obj.statementseq) return obj.statement.accept(this)
    return [...obj.statementseq.accept(this), ...obj.statement.accept(this)]
}
GenericVisitor.prototype.visitStatement = function (ctx) {
    return this.visitChildren(ctx)
}
GenericVisitor.prototype.visitExpressionstatement = function (ctx) {
    var obj = getGenericContext.apply(this, [ctx])
    return obj.expression.accept(this)
}
GenericVisitor.prototype.visitExpression = function (ctx) {
    var obj = getGenericContext.apply(this, [ctx])
    return this.visitChildren(ctx)[0]
}
GenericVisitor.prototype.visitAssignmentexpression = function (ctx) {
//    var obj = getGenericContext.apply(this, [ctx])
    return {
        type: 'AssignmentExpression',
        operator: '=' | '*=' | '**=' | '/=' | '%=' | '+=' | '-=' |
            '<<=' | '>>=' | '>>>=' | '&=' | '^=' | '|=',
        left: 0,
        right: this.visitChildren(ctx)[0],
    }
}
GenericVisitor.prototype.visitConditionalexpression = function (ctx) {
//    var obj = getGenericContext.apply(this, [ctx])
    return {
        type: 'ConditionalExpression',
        test: this.visitChildren(ctx)[0],
        consequent: 0,
        alternate: 0,
    }
}
GenericVisitor.prototype.visitInclusiveorexpression = function (ctx) {
//    var obj = getGenericContext.apply(this, [ctx])
    return {
        type: 'LogicalExpression',
        operator: '||',
        left: this.visitChildren(ctx),
        right: 0,
    }
}
GenericVisitor.prototype.visitExclusiveorexpression = function (ctx) {
    return {
        type: 'LogicalExpression',
        operator: '||',
        left: this.visitChildren(ctx),
        right: 0,
    }
}
GenericVisitor.prototype.visitRelationalexpression = function (ctx) {
    return this.visitChildren(ctx)[0]
}
GenericVisitor.prototype.visitAdditiveexpression = function (ctx) {
    return this.visitChildren(ctx)[0]
}
GenericVisitor.prototype.visitDeclarator = function (ctx) {
    var params = ctx.parametersandqualifiers()
    if(params) return params.accept(this)
    return {
        type: 'Identifier',
        name: getGenericToken(parser, ctx._start).value,
        body: this.visitChildren(ctx)
    }
}
GenericVisitor.prototype.visitDeclspecifierseq = function (ctx) {
//    console.log(ctx.attributespecifierseq())
//    console.log(ctx.declspecifierseq())
//    console.log(ctx.declspecifier())
    var seq = ctx.declspecifierseq()
    if(!seq) return ctx.declspecifier().accept(this)
    return [ctx.declspecifier().accept(this), seq.accept(this)]
}
GenericVisitor.prototype.visitDeclspecifier = function (ctx) {
    return {
        storage: ctx.storageclassspecifier(),
        type: ctx.typespecifier(),
        function: ctx.functionspecifier(),
        typedef: getGenericToken(ctx.Typedef()),
        friend: getGenericToken(ctx.Friend()),
        const: getGenericToken(ctx.Constexpr())
    }
}
GenericVisitor.prototype.visitPseudodestructorname = function (ctx) {
    //console.log('hit')
    return {
        id: ctx.decltypespecifier().accept(this)
    }
}
GenericVisitor.prototype.visitSimpledeclaration = function (ctx) {
    return {
        type: 'Declarator',
        value: getGenericToken(parser, ctx._start).value,
        body: this.visitChildren(ctx)
    }
}

var types = selectDom(['//SimpletypespecifierContext'], dom)
types.forEach(t => {
    var node = selectDom(['.//TerminalNode'], t)
    assert(node.length === 1)
    t.replaceWith(selectDom('./*', treeToHtml([{
        type: 'Type',
        id: node[0].getAttribute('value')
    }])))
})

var ids = selectDom(['//IdexpressionContext'], dom)
ids.forEach(i => {
    var node = selectDom(['.//TerminalNode'], i)
    // unary operator parses incorrectly
    var ctx = selectDom(['.//ClassnameContext'], i)
    if(ctx.length > 0 && node[0].getAttribute('value') === '~') {
        i.replaceWith(selectDom('./*', treeToHtml([{
            type: 'UnaryExpression',
            operator: '~',
            argument: {
                type: 'Identifier',
                name: node[1].getAttribute('value')
            },
            prefix: true
        }])))
    } else {
        assert(node.length === 1)
        i.replaceWith(selectDom('./*', treeToHtml([{
            type: 'Identifier',
            name: node[0].getAttribute('value')
        }])))
    }
})

// TODO: use transpile to get even more succinct
selectDom([
    '//MemberdeclarationContext',
    replace(ctx => selectDom('./*', treeToHtml([{
        type: 'Property',
        kind: htmlToTree(selectDom('.//Type', ctx)),
        key: htmlToTree(selectDom('.//Identifier', ctx)),
        method: false,
        computed: false,
        value: null,
        shorthand: false,
    }])))
], dom)

selectDom([
    // TODO: use parent:: method instead especially since it supports it
    '//ClassspecifierContext',
    replace(ctx => selectDom('./*', treeToHtml([{
        type: 'ClassExpression',
        id: null,
        superClass: selectDom('.//ClasskeyContext//@value', ctx),
        body: {
            type: 'ClassBody',
            body: htmlToTree(selectDom('.//Property', ctx))
        }
    }])))
], dom)

selectDom([
    '//DeclspecifierseqContext[not(.//DeclspecifierseqContext)]',
    replace(ctx => {
        var result = selectDom([
            `.//Type[not(.//parent::Identifier)]
            |.//Identifier[not(.//parent::ClassExpression)]
            |.//ClassExpression`
        ], ctx)
        console.log(result)
        return result
    })
], dom)

selectDom([
    '//SimpledeclarationContext',
    replace(ctx => selectDom('./*', treeToHtml([{
        type: 'VariableDeclaration',
        declarations: [{
            type: 'VariableDeclarator',
            id: htmlToTree(selectDom('.//Identifier', ctx)),
            init: htmlToTree(selectDom('.//ClassExpression', ctx)),
            kind: htmlToTree(selectDom('.//Type', ctx)),
            static: selectDom('.//DeclspecifierContext//@value', ctx) === 'static'
        }],
        kind: selectDom('.//CvqualifierContext//@value', ctx) === 'const'
            ? 'const'
            : selectDom('.//DeclspecifierContext//@value', ctx) === 'typedef'
                ? 'typedef'
                : 'let'
    }])))
], dom)


#### TODO: use g4 translator to automatically select entry point, because this is silly




#### TODO: convert antlr tree to json matching esprima format with text locations and tokens lists

antlr tree to es?


In [ ]:
var assert = require('assert')
var importer= require('../Core')
var {treeToHtml, htmlToTree} = importer.import(['tree to html', 'html to tree'])
var {selectDom} = importer.import('select tree')
var {transpile, replace} = importer.import('transpile code')

function antlrToESTree(tree) {
    console.log(tree)
    var dom = selectDom('./*', treeToHtml([tree]))
    
    selectDom([
        `//TerminalNode/ancestor::ThetypenameContext
        |//TerminalNode/ancestor::SimpletypespecifierContext`,
        replace(ctx => {
            var {range, value} = htmlToTree(selectDom('.//TerminalNode', ctx))
            return selectDom('./*', treeToHtml({
                type: 'Type',
                name: value,
                range: range,
            }))
        })
    ], dom)
    
    
    selectDom([
        `//TerminalNode/ancestor::IdexpressionContext`,
        replace(ctx => {
            if(selectDom('.//TerminalNode/ancestor::ClassnameContext', ctx)) {
                var {range, value} = htmlToTree(selectDom('.//TerminalNode[./parent::ClassnameContext]', ctx))
                return selectDom('./*', treeToHtml({
                    type: 'UnaryExpression',
                    operator: '~',
                    argument: {
                        type: 'Identifier',
                        name: value,
                        kind: null,
                        pointer: false,
                        range: range
                    },
                    prefix: true
                }))
            }
            var {range, value} = htmlToTree(selectDom('.//TerminalNode', ctx))
            return selectDom('./*', treeToHtml({
                type: 'Identifier',
                name: value,
                kind: null,
                pointer: false,
                range: range,
            }))
        })
    ], dom)
    
    selectDom([
        `//MemberdeclarationContext`,
        replace(ctx => {
            var {kind, key, semi} = selectDom({
                kind: './/Type',
                key: './/Identifier',
                semi: './/TerminalNode[@value = ";"]'
            }, ctx)
            var id = htmlToTree(key)
            id.kind = htmlToTree(kind)
            return selectDom('./*', treeToHtml({
                type: 'Property',
                kind: 'init',
                key: id,
                method: false,
                computed: false,
                value: null,
                shorthand: false,
                range: [
                    htmlToTree(kind).range[0],
                    htmlToTree(semi).range[1]
                ]
            }))
        })
    ], dom)
    
    selectDom([
        `//SimpledeclarationContext[
            .//ClassheadContext//TerminalNode[@value = "struct"]
        ]`,
        replace(ctx => {
            var {typedef, body, id, superClass, semi} = selectDom({
                typedef: './/TerminalNode[@value = "typedef"]',
                id: './/InitdeclaratorContext//Identifier',
                body: ['.//Property'],
                superClass: './/ClasskeyContext//@value',
                semi: './TerminalNode[@value = ";"]'
            }, ctx)
            return selectDom('./*', treeToHtml({
                type: 'ClassExpression',
                id: htmlToTree(id),
                superClass: superClass,
                body: {
                    type: 'ClassBody',
                    body: htmlToTree(body)
                },
                typedef: true,
                range: [
                    htmlToTree(typedef).range[0],
                    htmlToTree(semi).range[1]
                ]
            }))
        })
    ], dom)
    
    selectDom([
        `//ParameterdeclarationContext`,
        replace(ctx => {
            var {type, constant, ptr, id} = selectDom({
                type: './/Type',
                constant: './/TerminalNode[@value="const"]/@value',
                ptr: './/TerminalNode[@value="*"]/@value',
                id: './/Identifier'
            }, ctx)
            if(!id) return type
            var result = Object.assign(htmlToTree(id), {
                kind: htmlToTree(type),
                pointer: !!ptr
            })
            return selectDom('./*', treeToHtml(result))
        })
    ], dom)
    
    /*
    selectDom([
        `//SimpledeclarationContext`,
        replace(ctx => {
            var {type, constant, ptr, ids} = selectDom({
                static: './/TerminalNode[@value="static"]',
                type: './/Type',
                constant: './/TerminalNode[@value="const"]/@value',
                ptr: './/TerminalNode[@value="*"]/@value',
                ids: ['.//Identifier'],
                semi: './TerminalNode[@value = ";"]'
            }, ctx)
            var result = {
                type: 'VariableDeclaration',
                declarations: htmlToTree(ids).map(id => ({
                    type: 'VariableDeclarator',
                    id: Object.assign(id, {
                        kind: htmlToTree(type),
                        pointer: !!ptr
                    }),
                    init: null
                })),
                kind: constant ? 'const' : 'let',
                range: [
                    
                ]
            }
            return selectDom('./*', treeToHtml(result))
        })
    ], dom)
    */
    
    return htmlToTree(dom)
}

module.exports = antlrToESTree


#### lets not forget what an esprima tree looks


In [4]:
var importer = require('../Core')
var {selectAst} = importer.import('select code tree')
var {htmlToTree} = importer.import('html to tree')


var root = selectAst('//Program', `
const val = ''
`)
console.log(htmlToTree(root).body[0])

/*

expects
{
  type: 'FunctionDeclaration',
  id: { type: 'Identifier', name: 'myfunc', range: [ 10, 16 ] },
  params: [
    { type: 'Identifier', name: 'ptr', range: [Array] },
    { type: 'Identifier', name: 'val', range: [Array] }
  ],
  body: { type: 'BlockStatement', body: [], range: [ 27, 31 ] },
  generator: false,
  expression: false,
  async: false,
  range: [ 1, 31 ]
}
{
  type: 'Program',
  body: [
    {
      type: 'VariableDeclaration',
      declarations: [Array],
      kind: 'var',
      range: [Array]
    },
    {
      type: 'VariableDeclaration',
      declarations: [Array],
      kind: 'var',
      range: [Array]
    },
  ],
  sourcetype: 'script',
  range: [ 0, 2317 ],
  comments: [
    {
      type: 'Line',
      value: ' TODO: get parser by language lookup',
      range: [Array]
    },
  ],
  tokens: [
    { type: 'Keyword', value: 'var', range: [Array] },
    { type: 'Identifier', value: 'fs', range: [Array] },
    { type: 'Punctuator', value: '=', range: [Array] },
  ]
}
{
  type: 'VariableDeclaration',
  declarations: [
    {
      type: 'VariableDeclarator',
      id: [Object],
      init: [Object],
      range: [Array]
    }
  ],
  kind: 'const',
  range: [ 1, 15 ]
}
*/



importing select tree,tree to html,html to tree - 3 cells - patterns.ipynb[2],syntax.ipynb[14],syntax.ipynb[16]
{
  type: 'VariableDeclaration',
  declarations: [
    {
      type: 'VariableDeclarator',
      id: [Object],
      init: [Object],
      range: [Array]
    }
  ],
  kind: 'const',
  range: [ 1, 15 ]
}


### TODO: build a directed graph of all filesystem calls and automatically asyncify

Remove nodes where call is followed by a return or is last in the context block and last in the function, these nodes won't need a callback but the next level up will need to be called.

Use main and abort frame and entry from getAnimationFrame as the top of the tree/graph.s


### file extension to language



#### the code 

file extension to language?

ext to lang?

language from file extension?


In [ ]:
import path from 'path'

var ext_to_lang = {
    // extension:language
    'm': 'cpp',
    'c': 'cpp',
    'h': 'cpp',
    'cpp': 'cpp',
    'dtd': 'dtd',
    'htm': 'html',
    'html': 'html',
    'java': 'java',
    'js': 'javascript',
    'mysql': 'mysql',
    'pl': 'perl',
    'php': 'php',
    'py': 'python',
    'rb': 'ruby',
    'sql': 'sql',
    'xml': 'xml',
    'vbs': 'vbscript',
    'as':'actionscript',
    'abap':'abap',
    'a':'ada',
    'applescript':'applescript',
    'x86':'asm',
    'asm':'asm',
    'm68k':'m68k',
    'pic16':'pic16',
    'z80':'z80',
    'log':'apache',
    'list':'apt_sources',
    'asp':'asp',
    'aspx':'asp',
    'aut':'autoit',
    'abnf':'bnf',
    'bnf':'bnf',
    'sh':'bash',
    'bash':'bash',
    'bgm':'basic4gl',
    'bb':'blitzbasic',
    'b':'bf',
    'bf':'bf',
    'c':'cpp',
    'h':'cpp',
    'cpp':'cpp',
    'cs':'csharp',
    'csharp':'csharp',
    'dcl':'caddcl',
    'lsp':'cadlisp',
    'cfdg':'cfdg',
    'cil':'cil',
    'cob':'cobol',
    'cbl':'cobol',
    'cobol':'cobol',
    'css':'css',
    'd':'d',
    'pas':'delphi',
    'dpr':'delphi',
    'patch':'diff',
    'diff':'diff',
    'batch':'dos',
    'dot':'dot',
    'eff':'eiffel',
    'f77':'fortran',
    'f95':'fortran',
    'ftn':'fortran',
    '4gl':'genero',
    'fbl':'freebasic',
    'mo':'gettext',
    'glsl':'glsl',
    'gml':'gml',
    'plt':'gnuplot',
    'groovy':'groovy',
    'gs':'haskell',
    'hs':'haskell',
    'hq9+':'hq9plus',
    'htm':'html',
    'html':'html',
    'ini':'ini',
    'ino':'inno',
    'myp':'inno',
    'i':'intercal',
    'io':'io',
    'java':'java',
    'js':'javascript',
    'javascript':'javascript',
    'kix':'kixtart',
    'tex':'latex',
    'lisp':'lisp',
    'lsp':'lisp',
    'lols':'lolcode',
    'lol':'lolcode',
    'lsx':'lotusscript',
    'lscript':'lscript',
    'lua':'lua',
    'makefile':'make',
    'make':'make',
    'mrc':'mirc',
    'mxml':'mxml',
    'mysql':'mysql',
    'nsh':'nsis',
    'nsi':'nsis',
    'cma':'ocaml',
    'p':'pascal',
    'pl':'perl',
    'php':'php',
    'php4':'php',
    'php5':'php',
    'phps':'php',
    'pbk':'pixelbender',
    'pl':'plsql',
    'pov':'povray',
    'ps':'powershell',
    'pl':'prolog',
    'pvx':'providex',
    'py':'python',
    'qbasic':'qbasic',
    'rb':'ruby',
    'ruby':'ruby',
    'sas':'sas',
    'sas':'scala',
    'scm':'scheme',
    'sce':'scilab',
    'st':'smalltalk',
    'sql':'sql',
    'tk':'tcl',
    'tcl':'tcl',
    'tclx':'tcl',
    'tbasic':'thinbasic',
    'typo3':'typoscript',
    'idl':'idl',
    'vb':'vbnet',
    'v':'verilog',
    'vhdl':'vhdl',
    'vim':'vim',
    'wbt':'winbatch',
    'reg':'reg',
    'xml':'xml',
    'x++':'xpp',
    'vbs':'vb',
}

var lang_to_parser = [
  '../Resources/Parsers/abnf/Abnf.g4',
  '../Resources/Parsers/agc/agc.g4',
  '../Resources/Parsers/algol60/algol60.g4',
  '../Resources/Parsers/alpaca/alpaca.g4',
  '../Resources/Parsers/antlr/antlr3/ANTLRv3.g4',
  '../Resources/Parsers/antlr/antlr4/ANTLRv4Lexer.g4',
  '../Resources/Parsers/antlr/antlr4/ANTLRv4LexerPythonTarget.g4',
  '../Resources/Parsers/antlr/antlr4/ANTLRv4Parser.g4',
  '../Resources/Parsers/antlr/antlr4/LexBasic.g4',
  '../Resources/Parsers/antlr/antlr4/examples/CPP14.g4',
  '../Resources/Parsers/antlr/antlr4/examples/Hello.g4',
  '../Resources/Parsers/antlr/antlr4/examples/Issue1567.g4',
  '../Resources/Parsers/antlr/antlr4/examples/issue1165.g4',
  '../Resources/Parsers/antlr/antlr4/examples/issue1165_2.g4',
  '../Resources/Parsers/antlr/antlr4/examples/issue1165_3.g4',
  '../Resources/Parsers/antlr/antlr4/examples/issue1165_4.g4',
  '../Resources/Parsers/antlr/antlr4/examples/three.g4',
  '../Resources/Parsers/antlr/antlr4/examples/twocomments.g4',
  '../Resources/Parsers/apex/apex.g4',
  '../Resources/Parsers/arithmetic/arithmetic.g4',
  '../Resources/Parsers/asm/asm6502/asm6502.g4',
  '../Resources/Parsers/asm/asm8080/asm8080.g4',
  '../Resources/Parsers/asm/asm8086/asm8086.g4',
  '../Resources/Parsers/asm/asmMASM/asmMASM.g4',
  '../Resources/Parsers/asm/asmZ80/asmZ80.g4',
  '../Resources/Parsers/asm/masm/MASM.g4',
  '../Resources/Parsers/asn/asn/ASN.g4',
  '../Resources/Parsers/asn/asn_3gpp/ASN_3gpp.g4',
  '../Resources/Parsers/aspectj/AspectJLexer.g4',
  '../Resources/Parsers/aspectj/AspectJParser.g4',
  '../Resources/Parsers/atl/ATL.g4',
  '../Resources/Parsers/b/b.g4',
  '../Resources/Parsers/basic/jvmBasic.g4',
  '../Resources/Parsers/bnf/bnf.g4',
  '../Resources/Parsers/brainfuck/brainfuck.g4',
  '../Resources/Parsers/c/C.g4',
  '../Resources/Parsers/calculator/calculator.g4',
  '../Resources/Parsers/capnproto/CapnProto.g4',
  '../Resources/Parsers/clf/clf.g4',
  '../Resources/Parsers/clif/CLIF.g4',
  '../Resources/Parsers/clojure/Clojure.g4',
  '../Resources/Parsers/clu/clu.g4',
  '../Resources/Parsers/cmake/CMake.g4',
  '../Resources/Parsers/cobol85/Cobol85.g4',
  '../Resources/Parsers/cobol85/Cobol85Preprocessor.g4',
  '../Resources/Parsers/cookie/cookie.g4',
  '../Resources/Parsers/cool/COOL.g4',
  '../Resources/Parsers/cpp/CPP14.g4',
  '../Resources/Parsers/cql3/CqlLexer.g4',
  '../Resources/Parsers/cql3/CqlParser.g4',
  '../Resources/Parsers/creole/creole.g4',
  '../Resources/Parsers/csharp/CSharpLexer.g4',
  '../Resources/Parsers/csharp/CSharpParser.g4',
  '../Resources/Parsers/csharp/CSharpPreprocessorParser.g4',
  '../Resources/Parsers/csharp/CSharpSharwell/CSharpLexer.g4',
  '../Resources/Parsers/csharp/CSharpSharwell/CSharpPreprocessorParser.g4',
  '../Resources/Parsers/css3/css3.g4',
  '../Resources/Parsers/csv/CSV.g4',
  '../Resources/Parsers/cto/CtoLexer.g4',
  '../Resources/Parsers/cto/CtoParser.g4',
  '../Resources/Parsers/dart2/Dart2.g4',
  '../Resources/Parsers/databank/databank.g4',
  '../Resources/Parsers/dcm/DCM_2_0_grammar.g4',
  '../Resources/Parsers/dice/DiceNotation.g4',
  '../Resources/Parsers/dice/DiceNotationLexer.g4',
  '../Resources/Parsers/dot/DOT.g4',
  '../Resources/Parsers/edif300/EDIF300.g4',
  '../Resources/Parsers/erlang/Erlang.g4',
  '../Resources/Parsers/fasta/fasta.g4',
  '../Resources/Parsers/fen/fen.g4',
  '../Resources/Parsers/flatbuffers/FlatBuffers.g4',
  '../Resources/Parsers/fol/fol.g4',
  '../Resources/Parsers/fortran77/Fortran77Lexer.g4',
  '../Resources/Parsers/fortran77/Fortran77Parser.g4',
  '../Resources/Parsers/fusion-tables/FusionTablesSql.g4',
  '../Resources/Parsers/gff3/gff3.g4',
  '../Resources/Parsers/gml/gml.g4',
  '../Resources/Parsers/golang/GoLexer.g4',
  '../Resources/Parsers/golang/GoParser.g4',
  '../Resources/Parsers/graphql/GraphQL.g4',
  '../Resources/Parsers/graphstream-dgs/DGSLexer.g4',
  '../Resources/Parsers/graphstream-dgs/DGSParser.g4',
  '../Resources/Parsers/gtin/gtin.g4',
  '../Resources/Parsers/guido/guido.g4',
  '../Resources/Parsers/html/HTMLLexer.g4',
  '../Resources/Parsers/html/HTMLParser.g4',
  '../Resources/Parsers/http/http.g4',
  '../Resources/Parsers/hypertalk/HyperTalk.g4',
  '../Resources/Parsers/icalendar/ICalendar.g4',
  '../Resources/Parsers/icon/icon.g4',
  '../Resources/Parsers/idl/IDL.g4',
  '../Resources/Parsers/inf/inf.g4',
  '../Resources/Parsers/informix/informix.g4',
  '../Resources/Parsers/iri/IRI.g4',
  '../Resources/Parsers/istc/istc.g4',
  '../Resources/Parsers/java/java/JavaLexer.g4',
  '../Resources/Parsers/java/java/JavaParser.g4',
  '../Resources/Parsers/java/java8/Java8Lexer.g4',
  '../Resources/Parsers/java/java8/Java8Parser.g4',
  '../Resources/Parsers/java/java9/Java9.g4',
  '../Resources/Parsers/javadoc/JavadocLexer.g4',
  '../Resources/Parsers/javadoc/JavadocParser.g4',
  '../Resources/Parsers/javascript/ecmascript/CSharp/ECMAScript.g4',
  '../Resources/Parsers/javascript/ecmascript/CSharpSharwell/ECMAScript.g4',
  '../Resources/Parsers/javascript/ecmascript/ECMAScript.g4',
  '../Resources/Parsers/javascript/ecmascript/Go/ECMAScript.g4',
  '../Resources/Parsers/javascript/ecmascript/JavaScript/ECMAScript.g4',
  '../Resources/Parsers/javascript/ecmascript/Python/ECMAScript.g4',
  '../Resources/Parsers/javascript/javascript/JavaScriptLexer.g4',
  '../Resources/Parsers/javascript/javascript/JavaScriptParser.g4',
  '../Resources/Parsers/jpa/JPA.g4',
  '../Resources/Parsers/json/JSON.g4',
  '../Resources/Parsers/jsonquery/JsonQuery.g4',
  '../Resources/Parsers/kirikiri-tjs/TJSLexer.g4',
  '../Resources/Parsers/kirikiri-tjs/TJSParser.g4',
  '../Resources/Parsers/kotlin/kotlin/KotlinLexer.g4',
  '../Resources/Parsers/kotlin/kotlin/KotlinParser.g4',
  '../Resources/Parsers/kotlin/kotlin/UnicodeClasses.g4',
  '../Resources/Parsers/kotlin/kotlin-formal/KotlinLexer.g4',
  '../Resources/Parsers/kotlin/kotlin-formal/KotlinParser.g4',
  '../Resources/Parsers/kotlin/kotlin-formal/UnicodeClasses.g4',
  '../Resources/Parsers/kuka/krl.g4',
  '../Resources/Parsers/lambda/lambda.g4',
  '../Resources/Parsers/lcc/lcc.g4',
  '../Resources/Parsers/less/LessLexer.g4',
  '../Resources/Parsers/less/LessParser.g4',
  '../Resources/Parsers/logo/logo/logo.g4',
  '../Resources/Parsers/logo/ucb-logo/UCBLogo.g4',
  '../Resources/Parsers/lolcode/lolcode.g4',
  '../Resources/Parsers/lpc/LPC.g4',
  '../Resources/Parsers/lua/Lua.g4',
  '../Resources/Parsers/matlab/matlab.g4',
  '../Resources/Parsers/mdx/mdx.g4',
  '../Resources/Parsers/memcached_protocol/memcached_protocol.g4',
  '../Resources/Parsers/metric/metric.g4',
  '../Resources/Parsers/modelica/modelica.g4',
  '../Resources/Parsers/modula2pim4/m2pim4.g4',
  '../Resources/Parsers/molecule/molecule.g4',
  '../Resources/Parsers/moo/moo.g4',
  '../Resources/Parsers/morsecode/morsecode.g4',
  '../Resources/Parsers/mps/mps.g4',
  '../Resources/Parsers/muddb/muddb.g4',
  '../Resources/Parsers/mumath/mumath.g4',
  '../Resources/Parsers/mumps/mumps.g4',
  '../Resources/Parsers/muparser/MuParser.g4',
  '../Resources/Parsers/objc/ObjectiveCParser.g4',
  '../Resources/Parsers/objc/one-step-processing/ObjectiveCLexer.g4',
  '../Resources/Parsers/objc/one-step-processing/ObjectiveCPreprocessorParser.g4',
  '../Resources/Parsers/objc/two-step-processing/ObjectiveCLexer.g4',
  '../Resources/Parsers/objc/two-step-processing/ObjectiveCPreprocessorLexer.g4',
  '../Resources/Parsers/objc/two-step-processing/ObjectiveCPreprocessorParser.g4',
  '../Resources/Parsers/oncrpc/oncrpcv2.g4',
  '../Resources/Parsers/oncrpc/xdr.g4',
  '../Resources/Parsers/p/p.g4',
  '../Resources/Parsers/pascal/pascal.g4',
  '../Resources/Parsers/pcre/PCRE.g4',
  '../Resources/Parsers/pddl/Pddl.g4',
  '../Resources/Parsers/pdn/pdn.g4',
  '../Resources/Parsers/pdp7/pdp7.g4',
  '../Resources/Parsers/peoplecode/PeopleCode.g4',
  '../Resources/Parsers/pgn/PGN.g4',
  '../Resources/Parsers/php/PhpLexer.g4',
  '../Resources/Parsers/php/PhpParser.g4',
  '../Resources/Parsers/pike/pike.g4',
  '../Resources/Parsers/pl0/pl0.g4',
  '../Resources/Parsers/ply/ply.g4',
  '../Resources/Parsers/postalcode/postalcode.g4',
  '../Resources/Parsers/powerbuilder/PowerBuilderLexer.g4',
  '../Resources/Parsers/powerbuilder/PowerBuilderParser.g4',
  '../Resources/Parsers/prolog/prolog.g4',
  '../Resources/Parsers/propcalc/propcalc.g4',
  '../Resources/Parsers/properties/properties.g4',
  '../Resources/Parsers/protobuf3/Protobuf3.g4',
  '../Resources/Parsers/prov-n/PROV_N.g4',
  '../Resources/Parsers/python/python/PythonLexer.g4',
  '../Resources/Parsers/python/python/PythonParser.g4',
  '../Resources/Parsers/python/python2/Python2.g4',
  '../Resources/Parsers/python/python2-js/Python2.g4',
  '../Resources/Parsers/python/python3/Python3.g4',
  '../Resources/Parsers/python/python3-cs/Python3.g4',
  '../Resources/Parsers/python/python3-js/Python3.g4',
  '../Resources/Parsers/python/python3-py/Python3.g4',
  '../Resources/Parsers/python/python3-ts/Python3.g4',
  '../Resources/Parsers/python/python3_without_actions/Main/Python3.g4',
  '../Resources/Parsers/python/python3alt/AltPython3.g4',
  '../Resources/Parsers/quakemap/quakemap.g4',
  '../Resources/Parsers/r/R.g4',
  '../Resources/Parsers/r/RFilter.g4',
  '../Resources/Parsers/rcs/RCS.g4',
  '../Resources/Parsers/redcode/redcode.g4',
  '../Resources/Parsers/rego/RegoLexer.g4',
  '../Resources/Parsers/rego/RegoParser.g4',
  '../Resources/Parsers/restructuredtext/ReStructuredText.g4',
  '../Resources/Parsers/rexx/RexxLexer.g4',
  '../Resources/Parsers/rexx/RexxParser.g4',
  '../Resources/Parsers/rfc822/rfc822-datetime/datetime.g4',
  '../Resources/Parsers/rfc822/rfc822-emailaddress/emailaddress.g4',
  '../Resources/Parsers/robotwars/robotwar.g4',
  '../Resources/Parsers/romannumerals/romannumerals.g4',
  '../Resources/Parsers/rpn/rpn.g4',
  '../Resources/Parsers/ruby/Corundum.g4',
  '../Resources/Parsers/scala/Scala.g4',
  '../Resources/Parsers/scotty/scotty.g4',
  '../Resources/Parsers/scss/ScssLexer.g4',
  '../Resources/Parsers/scss/ScssParser.g4',
  '../Resources/Parsers/sexpression/sexpression.g4',
  '../Resources/Parsers/sgf/sgf.g4',
  '../Resources/Parsers/sharc/SHARCLexer.g4',
  '../Resources/Parsers/sharc/SHARCParser.g4',
  '../Resources/Parsers/sickbay/sickbay.g4',
  '../Resources/Parsers/smalltalk/Smalltalk.g4',
  '../Resources/Parsers/smiles/smiles.g4',
  '../Resources/Parsers/smtlibv2/SMTLIBv2.g4',
  '../Resources/Parsers/snobol/snobol.g4',
  '../Resources/Parsers/solidity/Solidity.g4',
  '../Resources/Parsers/sparql/Sparql.g4',
  '../Resources/Parsers/sql/mysql/Positive-Technologies/MySqlLexer.g4',
  '../Resources/Parsers/sql/mysql/Positive-Technologies/MySqlParser.g4',
  '../Resources/Parsers/sql/plsql/PlSqlLexer.g4',
  '../Resources/Parsers/sql/plsql/PlSqlParser.g4',
  '../Resources/Parsers/sql/sqlite/SQLite.g4',
  '../Resources/Parsers/sql/tsql/TSqlLexer.g4',
  '../Resources/Parsers/sql/tsql/TSqlParser.g4',
  '../Resources/Parsers/stacktrace/StackTrace.g4',
  '../Resources/Parsers/stellaris/stellaris.g4',
  '../Resources/Parsers/stringtemplate/LexBasic.g4',
  '../Resources/Parsers/stringtemplate/LexUnicode.g4',
  '../Resources/Parsers/stringtemplate/STGLexer.g4',
  '../Resources/Parsers/stringtemplate/STGParser.g4',
  '../Resources/Parsers/stringtemplate/STLexer.g4',
  '../Resources/Parsers/stringtemplate/STParser.g4',
  '../Resources/Parsers/suokif/SUOKIF.g4',
  '../Resources/Parsers/swift/swift2/Swift2.g4',
  '../Resources/Parsers/swift/swift3/Swift3.g4',
  '../Resources/Parsers/swift-fin/src/main/antlr4/SwiftFinLexer.g4',
  '../Resources/Parsers/swift-fin/src/main/antlr4/SwiftFinParser.g4',
  '../Resources/Parsers/telephone/telephone.g4',
  '../Resources/Parsers/thrift/Thrift.g4',
  '../Resources/Parsers/tiny/tiny.g4',
  '../Resources/Parsers/tinybasic/tinybasic.g4',
  '../Resources/Parsers/tinyc/tinyc.g4',
  '../Resources/Parsers/tinymud/tinymud.g4',
  '../Resources/Parsers/tnsnames/tnsnamesLexer.g4',
  '../Resources/Parsers/tnsnames/tnsnamesParser.g4',
  '../Resources/Parsers/tnt/tnt.g4',
  '../Resources/Parsers/toml/toml.g4',
  '../Resources/Parsers/tsv/tsv.g4',
  '../Resources/Parsers/turtle/TURTLE.g4',
  '../Resources/Parsers/turtle-doc/turtle.g4',
  '../Resources/Parsers/typescript/TypeScriptLexer.g4',
  '../Resources/Parsers/typescript/TypeScriptParser.g4',
  '../Resources/Parsers/unicode/graphemes/Graphemes.g4',
  '../Resources/Parsers/unicode/unicode16/classify.g4',
  '../Resources/Parsers/upnp/Upnp.g4',
  '../Resources/Parsers/url/url.g4',
  '../Resources/Parsers/useragent/useragent.g4',
  '../Resources/Parsers/v/V.g4',
  '../Resources/Parsers/vb6/VisualBasic6.g4',
  '../Resources/Parsers/vba/vba.g4',
  '../Resources/Parsers/verilog/system_verilog/SysVerilogHDL.g4',
  '../Resources/Parsers/verilog/verilog/Verilog2001.g4',
  '../Resources/Parsers/vhdl/vhdl.g4',
  '../Resources/Parsers/wat/WatLexer.g4',
  '../Resources/Parsers/wat/WatParser.g4',
  '../Resources/Parsers/wavefront/WavefrontOBJ.g4',
  '../Resources/Parsers/webidl/WebIDL.g4',
  '../Resources/Parsers/wkt/wkt.g4',
  '../Resources/Parsers/xml/XMLLexer.g4',
  '../Resources/Parsers/xml/XMLParser.g4',
  '../Resources/Parsers/xpath/xpath.g4',
  '../Resources/Parsers/xsd-regex/regexLexer.g4',
  '../Resources/Parsers/xsd-regex/regexParser.g4',
  '../Resources/Parsers/z/ZLexer.g4',
  '../Resources/Parsers/z/ZOperatorParser.g4',
  '../Resources/Parsers/z/ZParser.g4'
]

function extToLang(filename) {
    var result = ext_to_lang[filename]
        || filename.replace(/^\./, '')
        || ext_to_lang[path.extname(filename).replace(/^\./, '')]
    if(!result) throw new Error(`Unknown filetype ${filename}`)
    return result
}

export default extToLang


### automatically generate all parsers

Because why the heck not?



#### the code

generate antlr parsers?


In [ ]:
var path = require('path')
var importer = require('../Core')
var {glob} = importer.import('glob files')
var execCmd = importer.import('spawn child process')
var {chext} = importer.import('changing file name')

async function generateAntlr() {
    var files = glob('**/*.g4', path.join(__dirname, '../Resources/Parsers/'))

    /*
    for(var i = 0; i < files.length; i++) {
        try {
            await execCmd(`npx antlr4ts -visitor "${files[i]}"`)
        } catch (e) {
        }
        
        if(fs.existsSync(chext(files[i], '') + 'Parser.js')
           || fs.existsSync(chext(files[i], '.js'))) {
            continue
        }
        try {
            await execCmd(`java -Xmx500M -cp "/Users/briancullinan/Downloads/antlr-4.8-complete.jar:$CLASSPATH" org.antlr.v4.Tool -Dlanguage=JavaScript -visitor "${path.dirname(files[i])}"/*.g4`)
        } catch (e) {
        }
    }
    */
    
    var typescripts = glob('**/*.ts', path.join(__dirname, '../Resources/Parsers/'))
        .map(f => path.dirname(f))
        .filter((f, i, arr) => arr.indexOf(f) === i)
    
    for(var i = 0; i < typescripts.length; i++) {
        try {
            await execCmd(`npx babel --extensions .ts,.tsx "${typescripts[i]}" --out-dir "${typescripts[i]}"`)
        } catch (e) {
        }
    }
    
}

module.exports = generateAntlr
